# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [ ]:
%pip install -r ../requirements.txt

## Analyzer Templates

Below is a collection of analyzer templates designed to extract fields from various input file types.

These templates are highly customizable, allowing you to modify them to suit your specific needs. For additional verified templates from Microsoft, please visit [here](../analyzer_templates/README.md).

In [1]:
extraction_templates = {
    "invoice":            ('../analyzer_templates/invoice.json',         '../data/invoice.pdf'            ),
    "chart":              ('../analyzer_templates/image_chart.json',     '../data/pieChart.jpg'           ),
    "call_recording":     ('../analyzer_templates/call_recording_analytics.json', '../data/callCenterRecording.mp3'),
    "conversation_audio": ('../analyzer_templates/conversational_audio_analytics.json', '../data/callCenterRecording.mp3'),
    "marketing_video":    ('../analyzer_templates/marketing_video.json', '../data/FlightSimulator.mp4'              ),
    "call_recording_pretranscribe_batch": ('../analyzer_templates/call_recording_analytics_text.json', '../data/batch_pretranscribed.json'),
    "call_recording_pretranscribe_fast": ('../analyzer_templates/call_recording_analytics_text.json', '../data/fast_pretranscribed.json'),
    "call_recording_pretranscribe_cu": ('../analyzer_templates/call_recording_analytics_text.json', '../data/cu_pretranscribed.json')
}

Specify the analyzer template you want to use and provide a name for the analyzer to be created based on the template.

In [4]:
import uuid

# ANALYZER_TEMPLATE = "invoice"
ANALYZER_TEMPLATE = "call_recording_pretranscribe_batch"
ANALYZER_ID = "field-extraction-sample-" + str(uuid.uuid4())

(analyzer_template_path, analyzer_sample_file_path) = extraction_templates[ANALYZER_TEMPLATE]

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [2]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2024-12-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    # x_ms_useragent="azure-ai-content-understanding-python/field_extraction", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.17.1 Python/3.12.9 (Windows-11-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential


## Create Analyzer from the Template

In [5]:
response = client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=analyzer_template_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzer field-extraction-sample-24885356-78f0-4bca-9a60-e13b11b11831 create request accepted.
INFO:python.content_understanding_client:Request result is ready after 0.00 seconds.


{
  "id": "b84d8ce6-83a2-4cde-98c5-0eef3c17df96",
  "status": "Succeeded",
  "result": {
    "analyzerId": "field-extraction-sample-24885356-78f0-4bca-9a60-e13b11b11831",
    "description": "Sample call recording analytics",
    "createdAt": "2025-02-05T19:56:38Z",
    "lastModifiedAt": "2025-02-05T19:56:38Z",
    "config": {
      "returnDetails": false
    },
    "fieldSchema": {
      "fields": {
        "Summary": {
          "type": "string",
          "method": "generate",
          "description": "A one-paragraph summary"
        },
        "Topics": {
          "type": "array",
          "method": "generate",
          "description": "Top 5 topics mentioned",
          "items": {
            "type": "string"
          }
        },
        "Companies": {
          "type": "array",
          "method": "generate",
          "description": "List of companies mentioned",
          "items": {
            "type": "string"
          }
        },
        "People": {
          "type": "a

## Extract Fields Using the Analyzer

After the analyzer is successfully created, we can use it to analyze our input files.

In [ ]:
response = client.begin_analyze(ANALYZER_ID, file_location=analyzer_sample_file_path)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

In [6]:
#Use the following code to process pretranscribed files
from python.transcripts_processor import TranscriptsProcessor
test_file_path=analyzer_sample_file_path

transcripts_processor = TranscriptsProcessor()
webvtt_output, webvtt_output_file_path = transcripts_processor.convert_file(analyzer_sample_file_path)  

if "WEBVTT" not in webvtt_output:
    print("Error: The output is not in WebVTT format.")
else:    
    response = client.begin_analyze(ANALYZER_ID, file_location=webvtt_output_file_path)
    result = client.poll_result(response)

print(json.dumps(result, indent=2))


Load transcription completed.
processing a fast transcription file.
Fast to WebVTT Conversion completed.
Conversion completed. The result has been saved to '..\data\output\convertedTowebVTT.txt'


INFO:python.content_understanding_client:Analyzing file ..\data\output\convertedTowebVTT.txt with analyzer: field-extraction-sample-24885356-78f0-4bca-9a60-e13b11b11831
INFO:python.content_understanding_client:Request c1c8fbd0-75d4-4ce2-89fe-f03761eb4378 in progress ...
INFO:python.content_understanding_client:Request result is ready after 2.90 seconds.


{
  "id": "c1c8fbd0-75d4-4ce2-89fe-f03761eb4378",
  "status": "Succeeded",
  "result": {
    "analyzerId": "field-extraction-sample-24885356-78f0-4bca-9a60-e13b11b11831",
    "apiVersion": "2024-12-01-preview",
    "createdAt": "2025-02-05T19:56:59Z",
    "warnings": [],
    "contents": [
      {
        "markdown": "```\nWEBVTT\r\n\r\n00:00:00.080 --> 00:00:02.240\r\n<v Speaker 1>Hi there, thank you for calling Contoso.\r\n\r\n00:00:02.240 --> 00:00:03.200\r\n<v Speaker 1>My name is Sam.\r\n\r\n00:00:03.440 --> 00:00:04.720\r\n<v Speaker 1>How can I help you today?\r\n\r\n00:00:05.520 --> 00:00:07.680\r\n<v Speaker 2>Good afternoon, my name is Mary Adams.\r\n\r\n00:00:08.560 --> 00:00:10.720\r\n<v Speaker 2>I want to buy life insurance for my husband.\r\n\r\n00:00:11.520 --> 00:00:13.520\r\n<v Speaker 2>Can you please tell me how much it costs?\r\n\r\n00:00:14.400 --> 00:00:17.200\r\n<v Speaker 1>I'm going to help you right now with your life insurance needs.\r\n\r\n00:00:18.080 --> 0

## Clean Up
Optionally, delete the sample analyzer from your resource. In typical usage scenarios, you would analyze multiple files using the same analyzer.

In [ ]:
client.delete_analyzer(ANALYZER_ID)